# Normalization for batch effect

In this notebook we test different approaches of scaling and centering the feature table in order to minimize batch effect. 

load libraries

In [1]:
library(dplyr)
library(tidyr)
sessionInfo()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




R version 4.1.1 (2021-08-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur 10.16

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tidyr_1.1.3 dplyr_1.0.7

loaded via a namespace (and not attached):
 [1] magrittr_2.0.1   tidyselect_1.1.1 uuid_0.1-4       R6_2.5.1        
 [5] rlang_0.4.11     fastmap_1.1.0    fansi_0.5.0      tools_4.1.1     
 [9] utf8_1.2.2       DBI_1.1.1        htmltools_0.5.2  ellipsis_0.3.2  
[13] assertthat_0.2.1 digest_0.6.27    tibble_3.1.4     lifecycle_1.0.0 
[17] crayon_1.4.1     IRdisplay_1.0    purrr_0.3.4      repr_1.1.3      
[21] base64enc_0.1-3  vctrs_0.3.8      IRkernel_1.2     glue_1

load feature table

In [2]:
ft <- read.table('../../output/MS2_timsTOF_quant_mergedAdducts.csv', sep = ',', check.names = F, header = T, row.names = 1)

load metadata

In [3]:
md <- read.table('../../data/metadata/Metadata_timsTOF.txt', sep = '\t', header = T, comment.char = '', check.names = F)

In [4]:
rownames(md) <- as.character(md$filename)

In [5]:
ft <- t(ft)
ft <- ft[-c(1:9),]
rownames(ft) <- gsub(' cropped Peak area','',rownames(ft))

In [6]:
class(ft)<-"numeric"

In [7]:
if (length(which(is.na(rowSums(ft) == T))) != 0){
    ft <- ft[-which(is.na(rowSums(ft) == T)),]
}

In [8]:
dim(ft)

[1]  252 2068

In [9]:
which(is.na(rowSums(ft) == T))
which(is.na(colSums(ft) == T))

named integer(0)

named integer(0)

In [10]:
grep('Blank',rownames(ft))

[1] 220 228 251 252

remove blank features <br>

In [11]:
length(rownames(ft)[grep('sample',rownames(ft))])

[1] 200

In [12]:
length(md$filename[grep('sample',md$filename)])

[1] 200

get an overview of number of samples, blanks, pools etc. per plate

In [13]:
md[,which(colnames(md) %in% c('SampleType','plate'))] %>% 
gather (SampleType, plate) %>% 
group_by(SampleType, plate) %>% 
summarise(no = n()) %>% 
spread (SampleType, no)

`summarise()` has grouped output by 'SampleType'. You can override using the `.groups` argument.



plate,Blank,EC,PB,Pool,Sample
<int>,<int>,<int>,<int>,<int>,<int>
1,2,8,5,4,66
2,NA,8,4,4,67
3,2,8,4,4,67


exclude blank samples (solvent), to substract blank features from samples we are using the paper blanks 

In [14]:
ft <- ft[-grep('Blank',rownames(ft)),]

In [15]:
blankmeans <- colMeans(ft[grep('PB',rownames(ft)),])

In [16]:
blankids <- grep('PB',rownames(ft))

remove features in samples if they are less than 20 times as intense as in the paper blank samples

In [17]:
for (i in 1:ncol(ft)){
    ft[-blankids,i][which(ft[-blankids,i] < 20*blankmeans[i])] <- 0
}

In [18]:
if (length(which(colSums(ft) == 0)) != 0){
    ft <- ft[,-which(colSums(ft) == 0)]
}

check whether filenames in feature table and metadata are identical

In [19]:
md <- md[match(rownames(ft),md$filename),]
identical(as.character(md$filename),as.character(rownames(ft)))

[1] TRUE

In [20]:
table(md$SampleType)


    EC     PB   Pool Sample 
    24     12     12    200 

In [21]:
dim(md)

[1] 248  28

In [22]:
dim(ft)

[1]  248 1986

# OPTIONAL: only selecte samples, not blanks and controls

In [23]:
identical(as.character(md$filename),as.character(rownames(ft)))

[1] TRUE

In [24]:
unique(md$SampleType)

[1] "EC"     "Sample" "PB"     "Pool"

In [25]:
ft <- ft[-which(md$SampleType%in% c('PB','EC','Pool')),]

In [26]:
dim(ft)

[1]  200 1986

In [27]:
md <- md[-which(md$SampleType %in% c('PB','EC','Pool')),]

In [28]:
dim(md)

[1] 200  28

In [29]:
identical(as.character(md$filename),as.character(rownames(ft)))

[1] TRUE

remove all zero columns

In [30]:
if (length(which(colSums(ft)==0)) != 0){
    ft <- ft[,-which(colSums(ft)==0)]
}

In [31]:
dim(ft)

[1]  200 1285

save feature table containing samples only

In [32]:
write.table(ft,'output/FeatureTable_SamplesOnly.txt', sep = '\t', quote = F)

#### OPTIONAL filter for 0s

remove features which have NA values for more than 5% of the samples

In [33]:
cutoff <- 5

In [34]:
ft <- ft[,which(apply(ft, 2, function(c)sum(c!=0)) > (nrow(ft)/100)*cutoff)]

In [35]:
dim(ft)

[1] 200 731

# replace all 0's by NAs

In [36]:
ft[ft==0] <- NA

In [37]:
dim(ft)

[1] 200 731

test different scaling approaches 

In [38]:
ft_scaledTT <- scale(ft, center = T, scale = T)

In [39]:
ft_scaledFT <- scale(ft, center = F, scale = T)
ft_scaledTF <- scale(ft, center = T, scale = F)
ft_scaledFF <- scale(ft, center = F, scale = F)

scale feature table per batch

In [40]:
identical(rownames(ft), rownames(md))

[1] TRUE

In [41]:
batchfts <- split( as.data.frame(ft) , f = md$plate)

In [42]:
length(batchfts)

[1] 3

In [43]:
batchft_TT <- lapply( batchfts, scale, center = T, scale = T)
batchft_TT <- do.call("rbind", batchft_TT)

In [44]:
batchft_TF <- lapply( batchfts, scale, center = T, scale = F)
batchft_TF <- do.call("rbind", batchft_TF)
batchft_FT <- lapply( batchfts, scale, center = F, scale = T)
batchft_FT <- do.call("rbind", batchft_FT)
batchft_FF <- lapply( batchfts, scale, center = F, scale = F)
batchft_FF <- do.call("rbind", batchft_FF)

In [45]:
batchft_TT <- batchft_TT[match(rownames(md),rownames(batchft_TT)),]
batchft_TF <- batchft_TF[match(rownames(md),rownames(batchft_TF)),]
batchft_FT <- batchft_FT[match(rownames(md),rownames(batchft_FT)),]
batchft_FF <- batchft_FF[match(rownames(md),rownames(batchft_FF)),]

In [46]:
identical(rownames(batchft_TT),rownames(md))
identical(rownames(batchft_TF),rownames(md))
identical(rownames(batchft_FT),rownames(md))
identical(rownames(batchft_FF),rownames(md))

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

In [47]:
md$plate <- as.factor(md$plate)

In [48]:
md$year <- as.factor(md$year)

In [49]:
dim(ft)

[1] 200 731

## Assess batch effect using PERMANOVA and visualization in PCoA plots

load libraries

In [50]:
suppressMessages(library(vegan))
suppressMessages(library(ggplot2))
suppressMessages(library(gridExtra))
suppressMessages(library(ggsci))
source('sourcefunctions/plot_multiple_PcoAs.R')
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur 10.16

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggsci_2.9       gridExtra_2.3   ggplot2_3.3.5   vegan_2.5-7    
[5] lattice_0.20-44 permute_0.9-5   tidyr_1.1.3     dplyr_1.0.7    

loaded via a namespace (and not attached):
 [1] pillar_1.6.2     compiler_4.1.1   base64enc_0.1-3  tools_4.1.1     
 [5] digest_0.6.27    uuid_0.1-4       gtable_0.3.0     nlme_3.1-152    
 [9] jsonlite_1.7.2   evaluate_0.14    lifecycle_1.0.0  tibble_3.1.4    
[13] mgcv_1.8-36      pkgconfig_2.0.3  rlang_0.4.11     Matrix_1.3-4    
[17] IRdisplay_1.0    DBI_1

In [51]:
batches <- c(list(ft), 
            list(ft_scaledTT),  list(ft_scaledFT),
            list(ft_scaledTF),list(ft_scaledFF),
            list(batchft_TT),list(batchft_TF),
            list(batchft_FT),list(batchft_FF)) 

In [52]:
titles <- c('No batch correction', 'Scaled and centered', 'scaled', 'centered', 'none',
           'scaled and centered per batch', 'centered per batch', 'scaled per batch', 'none')

In [53]:
plot_PcoA(batches, md, distmetric = c("canberra"), collow = "#810f7c", colhigh ="#f7fcb9", cat = "plate", catcols = "SampleType", mdtype = 'categorical', cols = c('grey','black','orange','pink','darkgreen','darkred','brown','blue','skyblue3','green'), titles = titles, path_plots = ".", name_plot = "output/BatchCorr_summary_plot_5_NA.pdf")

pdf 
  2